In [2]:
# 1. Import libraries
import gspread
import pandas as pd
import hashlib
import os
import json
import numpy as np
import hmac
import calendar
import requests
from google.oauth2 import service_account
from datetime import datetime, timezone, timedelta,timezone
import toml

# 2. Load Google Cloud credentials from .streamlit/secrets.toml
with open(".streamlit/secrets.toml", "r") as f:
    secrets = toml.load(f)

google_cloud_secrets = secrets["google_cloud"]

# Setup credentials
creds = service_account.Credentials.from_service_account_info(
    google_cloud_secrets,
    scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
)

client = gspread.authorize(creds)

# 3. Setup Accurate credentials
api_token = "aat.NTA.eyJ2IjoxLCJ1Ijo4MTk2MTksImQiOjEyMjU0NDUsImFpIjo1NTA3NCwiYWsiOiJjMDU1MTNjZS02ZWJlLTRmZTAtYWQwNC00MGQ1NDQ1OWZmMDEiLCJhbiI6IktyaXN0YWwgQXV0b21hdGlvbiIsImFwIjoiNjQ5YTUyZGItOGY3Ni00NGQyLThmMGQtZGU2MDIwMDUwZGI4IiwidCI6MTc0NDc4MDU0NzcxMH0.QeoDo2DCNvI0V/fVQT2IQP8I6v69GOVMRRjSznGkLXdysVtbR9ZTky2FO45cFgVfwWPlLeeSpYXQrICLExUB0MYzX+BqgtXmm/Eb7vc+lRssKuhUbUdpYomW3WXLQSBUJeiPr5KT1wxaIBOk2PIBH09JqdY5yD94rG56xvy9urlr8km0HbtFeVlf8ScL6zFE/jJv4Nu+njY=.0xJjBtaQRlBRfXntN6r+aITBX6BFwvMLz5IK+eDUrpY"
signature_secret = "EjESUUVVTg5XYDUF9uzMB3PqgEH2G2Sj4OY54GA3k0QFTkb9J5hktE40RM1heIsf"

# 4. Function untuk membuat signature Accurate
def generate_signature(secret_key, timestamp):
    payload = timestamp.encode('utf-8')
    secret_key = secret_key.encode('utf-8')
    hmac_object = hmac.new(secret_key, payload, hashlib.sha256)
    return hmac_object.hexdigest()

# 5. Fungsi request ke API Accurate
def url(endpoint):
    host = "https://zeus.accurate.id"
    full_url = f"{host}{endpoint}"
    timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    signature = generate_signature(signature_secret, timestamp)

    headers = {
        "Authorization": f"Bearer {api_token}",
        "X-Api-Timestamp": timestamp,
        "X-Api-Signature": signature,
        "X-Language-Profile": "US"
    }

    response = requests.get(full_url, headers=headers)
    return response.json()

# 6. Fungsi ambil data dari Google Sheets
def get_data_gsheet(spreadsheet_id, sheetname, range_):
    all_data = client.open_by_key(spreadsheet_id).worksheet(sheetname).get(range_)
    headers = all_data[0]
    rows = all_data[1:]
    data = [dict(zip(headers, row)) for row in rows]
    return data

# 7. Fungsi update data ke Google Sheets
def update_data(spreadsheet_id, sheetname, df):
    sh = client.open_by_key(spreadsheet_id)
    worksheet = sh.worksheet(sheetname)
    worksheet.clear()
    data = [df.columns.tolist()] + df.values.tolist()
    data = [[cell if pd.notna(cell) else "" for cell in row] for row in data]
    worksheet.update('A1', data, value_input_option='USER_ENTERED')

# 8. Fungsi untuk ambil seluruh list data dari Accurate (multi-page)
def get_all_listdo(endpoint):
    page = 1
    all_data = []
    while True:
        data = url(f"{endpoint}&sp.page={page}")
        datas = data.get('d', [])
        if not datas:
            print(f"✅ Halaman {page} kosong, selesai.")
            break
        all_data.extend(datas)
        page += 1
    return all_data

# 9. Fungsi ambil detail journal voucher
def ambil_detail_journal(data):
    hasil = []
    for item in data:
        journal_id = item['id']
        detail = url(f"/accurate/api/journal-voucher/detail.do?id={journal_id}")
        for line in detail.get('d', {}).get('detailJournalVoucher', []):
            hasil.append({
                'id': journal_id,
                'date': item['transDate'],
                'type': line.get('amountType'),
                'akun': line.get('glAccount', {}).get('name'),
                'nilai': line.get('amount'),
                'description': line.get('description'),
                'memo': line.get('memo')
            })
    return hasil

today = datetime.today()
first_date = today - timedelta(days=14)
first_date = first_date.strftime("%d/%m/%Y")
last_date = today.strftime("%d/%m/%Y")
print(first_date)
print(last_date)

27/05/2025
10/06/2025


In [ ]:
update_json = get_all_listdo(f"/accurate/api/journal-voucher/list.do?fields=id,transDate&sp.pageSize=1000000")
update_json_journal = ambil_detail_journal(update_json)
df_json_jurnal = pd.DataFrame(update_json_journal)
df_json_jurnal['date'] = pd.to_datetime(df_json_jurnal['date'], errors='coerce', format="%d/%m/%Y")
df_json_jurnal['date'] = df_json_jurnal['date'].dt.strftime('%-m/%-d/%Y')
update_data("1cjR6k-OCWmeSfCRS_3b9B3wgQc8a-d_pIvWzcZpz1Uw","JSON",df_json_jurnal)

In [10]:
df = pd.DataFrame(get_data_gsheet("1cjR6k-OCWmeSfCRS_3b9B3wgQc8a-d_pIvWzcZpz1Uw", "JSON", "A:H"))

In [11]:
df['date'] = pd.to_datetime(df['date'])
df['nilai'] = pd.to_numeric(df['nilai'])

In [12]:
result = df.copy()

In [13]:
result['Debit'] = result.apply(lambda x: x['nilai'] if x['type'] == 'DEBIT' else 0, axis=1)
result['Credit'] = result.apply(lambda x: x['nilai'] if x['type'] == 'CREDIT' else 0, axis=1)
final_google = result[result['akun'].str.contains("Driver")]
final_google[final_google['akun'] == 'Driver - Antoni']
final_google = result[result['akun'].str.contains("Driver")]
final_google = final_google.sort_values('date')
final_google['Balance'] = (
final_google['Debit'] - final_google['Credit']  # pakai 'Kredit' jika sudah rename
).groupby(final_google['akun']).cumsum()
final_google['Saldo'] = np.where(final_google['Debit'] == 0, final_google['Credit'] * -1, 
                                        np.where(final_google['Credit'] == 0, final_google['Debit'], np.nan))

# Fill NaN values if needed (optional)
final_google['Saldo'] = final_google['Saldo'].fillna(0)
final_google['Driver Name'] = final_google['akun'].str.replace('Driver - ', '', regex=False)
final_google['date'] = final_google['date'].dt.strftime('%m/%d/%Y')
final_google = final_google[['akun', 'Driver Name', 'date', 'memo', 'Debit', 'Credit', 'Balance', 'Saldo']]
final_google = final_google.fillna("")
final_google.rename(columns={'Credit': 'Kredit', 'akun':'Nama Perkiraan', 'date':'Tanggal', 'memo': 'Deskripsi'}, inplace=True)
update_data("1cjR6k-OCWmeSfCRS_3b9B3wgQc8a-d_pIvWzcZpz1Uw","Valentio",final_google)

C:\Users\steph\AppData\Local\Temp\ipykernel_17488\2977667446.py:72: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data, value_input_option='USER_ENTERED')
